# Seed 생성 확인용 테스트 노트북

dbt seed로 생성된 seed 테이블 확인용

In [1]:
import sys
sys.path.append('/app')

from src.utils.spark_builder import get_spark_session
from pyspark.sql import functions as F

# Spark 세션 생성
spark = get_spark_session("GDELT_Seed_Test", "spark://spark-master:7077")
print("✅ Spark 세션 생성 완료")

25/09/11 06:20:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark 세션 생성 완료


In [2]:
# 1. Spark Metastore에 존재하는 모든 데이터베이스(스키마) 목록을 확인합니다.
print("📖 사용 가능한 데이터베이스(스키마) 목록:")
spark.sql("SHOW DATABASES").show()

📖 사용 가능한 데이터베이스(스키마) 목록:
+---------+
|namespace|
+---------+
|  default|
|     gold|
|   silver|
+---------+



In [3]:
# 2. 'default' 스키마 안에 있는 테이블 목록을 확인합니다.
# dbt seed로 생성된 테이블이 여기에 보여야 합니다. (gdelt_silver_events는 seed 테이블이 아닙니다.)
print("📜 'default' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN default").show()

📜 'default' 스키마의 테이블 목록:


+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|actor_ethnic_grou...|      false|
|  default|actor_organizatio...|      false|
|  default|actor_religion_codes|      false|
|  default|    actor_role_codes|      false|
|  default|  event_detail_codes|      false|
|  default|event_quad_class_...|      false|
|  default|    event_root_codes|      false|
|  default| gdelt_silver_events|      false|
|  default|       geo_adm_codes|      false|
|  default|geo_country_fips_...|      false|
|  default|geo_country_iso_c...|      false|
|  default|      geo_type_codes|      false|
+---------+--------------------+-----------+



In [4]:
# 3. 총 11개의 seed 테이블의 샘플 데이터를 5개씩 출력하여 내용 확인
seed_tables = [
    "actor_ethnic_group_codes",
    "actor_organization_codes",
    "actor_religion_codes",
    "actor_role_codes",
    "event_root_codes",
    "event_detail_codes",
    "event_quad_class_codes",
    "geo_adm_codes",
    "geo_country_fips_codes",
    "geo_country_iso_codes",
    "geo_type_codes"
]

for table in seed_tables:
    print(f"\n 🔍 {table} 테이블 샘플 데이터")
    try:
        spark.sql(f"SELECT * FROM default.{table} LIMIT 5").show(truncate=False)
    except Exception as e:
        print(f"❌ 테이블 조회 중, 오류 발생: {e}")


 🔍 actor_ethnic_group_codes 테이블 샘플 데이터


25/09/11 06:21:45 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/09/11 06:21:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+----+-----------------------------------+
|code|description                        |
+----+-----------------------------------+
|abk |Abkhaz (Abkhazians)                |
|abr |Aboriginal-Australians (Aborigines)|
|ace |Acehnese (Achinese)                |
|acg |Achang                             |
|ach |Acholi                             |
+----+-----------------------------------+


 🔍 actor_organization_codes 테이블 샘플 데이터
+----+-----+---------------------------+
|code|type |description                |
+----+-----+---------------------------+
|ACC |IGO  |Arab Cooperation Council   |
|ADB |IGO  |Asian Development Bank     |
|AEU |IGO  |Arab Economic Unity Council|
|AFB |IGO  |African Development Bank   |
|ALC |Party|All Liberia Coalition Party|
+----+-----+---------------------------+


 🔍 actor_religion_codes 테이블 샘플 데이터
+----+------------+---------------------+
|code|type        |description          |
+----+------------+---------------------+
|REL |Generic     |Unspecified Religious

생성된 seed 테이블을 삭제할 경우, 아래 코드에 "삭제할 seed 테이블명"을 입력하여 실행

In [ ]:
# 삭제할 seed 테이블 이름 목록
seed_tables_to_drop = [
    # "[삭제할 seed 테이블명]"
]
print("🗑️ 기존 Seed 테이블 삭제를 시작합니다.")

for table_name in seed_tables_to_drop:
    try:
        spark.sql(f"DROP TABLE IF EXISTS default.{table_name}")
        print(f"  - 테이블 'default.{table_name}' 삭제 완료.")
    except Exception as e:
        print(f"  - 테이블 'default.{table_name}' 삭제 중 오류 발생: {e}")

print("\n✅ 모든 Seed 테이블 삭제 작업이 완료되었습니다.")

# 삭제 후 테이블 목록을 다시 확인하여 깨끗해졌는지 확인
print("📜 현재 'default' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN default").show()

Staging 폴더의 stg_seed_mapping.sql을 dbt run한 결과 확인

In [20]:
# stg_seed_mapping 모델 결과 확인
print("🥈 [stg_seed_mapping] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM silver.stg_seed_mapping LIMIT 20").show(truncate=False)

🥈 [stg_seed_mapping] 뷰 : 상위 20개 데이터


+---------------+----------+-------------+--------------------------------+---------------------------------------------+------------------+---------------+------------+-----------+------------+-----------------+-----------+--------------+-----------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------------------------+-------------------+------------------+--------------------------+--------------------+--------------+---------------+---------------------+-----------+--------------+------------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------------------------------------------------------------------------+-------------------+------------------+--------------------------+--------------------+--------------+---------------+---------------------+------------------+-----------------------+--------------

Staging 폴더의 stg_seed_actors_parsed.sql을 dbt run한 결과 확인

In [17]:
# stg_seed_actors_parsed 모델 결과 확인
print("🥈 [stg_seed_actors_parsed] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM silver.stg_seed_actors_parsed LIMIT 20").show(truncate=False)

🥈 [stg_seed_actors_parsed] 뷰 : 상위 20개 데이터
+------------+--------------------------------------------------------------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+--------+----------------------------------------------+------------+------------------------------------------------------------------+-----+
|actor_code  |description1                                                                                                                          |type1   |description2                                                                    |type2   |description3                                  |type3       |description4                                                      |type4|
+------------+--------------------------------------------------------------------------------------------------------------------------------------+--------+------------------------------------

Marts 폴더의 mart_actor_info_test.sql을 dbt run한 결과 확인

In [18]:
# mart_actor_info_test 모델 결과 확인
print("🥇 [mart_actor_info_test] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM gold.mart_actor_info_test LIMIT 20").show(truncate=False)

🥇 [mart_actor_info_test] 뷰 : 상위 20개 데이터
+---------------+---------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+
|global_event_id|actor1_info                                                                      |actor2_info                                                                           |
+---------------+---------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+
|1262473778     |Occupied Palestinian Territory                                                   |Tunisia                                                                               |
|1262474211     |Elites: former government officials, celebrities, spokespersons for organizations|null                                                                                  |
|1262474699     |Nepal Po

Marts 폴더의 gold_gdelt_daily_summary.sql을 dbt run한 결과 확인

In [21]:
# gold_gdelt_daily_summary 모델 결과 확인
print("🥇 [gold_gdelt_daily_summary] 뷰")
spark.sql("SELECT * FROM gold.gold_gdelt_daily_summary").show(truncate=False)

🥇 [gold_gdelt_daily_summary] 뷰


+----------+------------+--------------+-------------+--------------+-------------------+-----------------+-------------------+------------------+---------+---------+---------+---------+---------+----------------+--------------------+-------------------------+---------------------------+--------------------+-------------------+--------------------------+
|event_date|events_total|articles_total|sources_total|mentions_total|avg_tone_mean      |avg_tone_median  |goldstein_mean     |goldstein_abs_mean|neg_ratio|quad1_cnt|quad2_cnt|quad3_cnt|quad4_cnt|top_country_name|top_country_articles|dq_action_geo_alpha_ratio|dq_action_geo_numeric_ratio|dq_actor1_iso3_ratio|dq_latlong_coverage|updated_at                |
+----------+------------+--------------+-------------+--------------+-------------------+-----------------+-------------------+------------------+---------+---------+---------+---------+---------+----------------+--------------------+-------------------------+--------------------------

In [ ]:
# spark.stop()
print("\n✅ Spark 세션이 종료되었습니다.")